# Results of first case study

In [ ]:
import numpy as np
import pandas as pd
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting import GroupNormalizer
from pytorch_forecasting import TemporalFusionTransformer, QuantileLoss
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import LearningRateMonitor
import params
import torch

In [15]:
time_df = pd.read_csv('dataframe.csv', index_col=0, sep=";", decimal=".")
time_df = time_df.reset_index(drop=True)
time_df["is_holiday_or_weekend"] = time_df["is_holiday_or_weekend"].replace(
    {True: 1, False: 0}
)
pattern = r"\d{1,2}\.?\s*[A-Za-z]{3}"
time_df = time_df.replace(pattern, float("nan"), regex=True)
time_df = time_df.ffill(axis=0)
time_df["Dayahead Prices Germany/Luxembourg"] = time_df[
    "Dayahead Prices Germany/Luxembourg"
].astype("float")

training_cutoff = time_df[params.time_idx].max() - params.max_prediction_length
training = TimeSeriesDataSet(
    time_df[lambda x: x.hours_from_start <= training_cutoff],
    time_idx=params.time_idx,
    target=params.target,
    group_ids=params.group_ids,
    min_encoder_length=params.max_encoder_length // 2,
    max_encoder_length=params.max_encoder_length,
    min_prediction_length=params.min_prediction_length,
    max_prediction_length=params.max_prediction_length,
    static_categoricals=params.static_categoricals,
    time_varying_known_reals=params.time_varying_known_reals,
    time_varying_unknown_reals=params.time_varying_unknown_reals,
    target_normalizer=GroupNormalizer(groups=["zone"], transformation="softplus"),
    add_relative_time_idx=params.add_relative_time_idx,
    add_target_scales=params.add_target_scales,
    add_encoder_length=params.add_encoder_length,
    allow_missing_timesteps=True,
)
validation = TimeSeriesDataSet.from_dataset(
    training, time_df, predict=True, stop_randomization=True
)

# if you have a strong GPU, feel free to increase the number of workers
train_dataloader = training.to_dataloader(
    train=True, batch_size=params.batch_size, num_workers=0
)
val_dataloader = validation.to_dataloader(
    train=False, batch_size=params.batch_size * 10, num_workers=0
)


C:\Users\Anwender\AppData\Local\Temp\ipykernel_23488\2580056126.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  time_df["is_holiday_or_weekend"] = time_df["is_holiday_or_weekend"].replace(


In [ ]:
# Example (you must use your actual model parameters!)
best_tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=params.learning_rate,
    hidden_size=params.hidden_size,
    attention_head_size=params.attention_head_size,
    dropout=params.dropout,
    hidden_continuous_size=params.hidden_continuous_size,
    output_size=params.output_size,  # there are 7 quantiles by default: [0.02, 0.1, 0.25, 0.5, 0.75, 0.9, 0.98]
    loss=QuantileLoss(),
    log_interval=params.log_interval,
    reduce_on_plateau_patience=params.reduce_on_plateau_patience,
)

# Load state dict
best_tft.load_state_dict(torch.load("best_tft_cpu_state_dict.pt", map_location="cpu"))